In [1]:
import time, threading, queue, sys, gc
from typing import Tuple

import numpy as np
import pyqtgraph as pg
from PyQt6 import QtCore
from collections import deque

import logging, logging.handlers

import h5py, pathlib
from datetime import datetime

| **速度目標 [rps]** | **1秒当たり [pulses / s]** | **1iterあたり [pulses]** | **カウント** | **再構成速度 (UP / DOWN) \[rps]** | **偏差 \[%]** |
| ----------------- | -------------------------- | --------------------------- | ----------------------- | ------------------------------- | -------------------------- |
| 0.1               | 204.8                      | 25.6                        | 26<br>25                | 0.102<br>0.09765625             | +1.56<br>−2.34             |
| 0.2               | 409.6                      | 51.2                        | 52<br>51                | 0.203<br>0.19921875             | +1.56<br>−0.39             |
| 0.3               | 614.4                      | 76.8                        | 77<br>76                | 0.301<br>0.296875               | +0.26<br>−1.04             |
| 0.4               | 819.2                      | 102.4                       | 103<br>102              | 0.402<br>0.3984375              | +0.59<br>−0.39             |
| 0.5               | 1 024.0                    | 128.0                       | 128<br>128              | 0.500<br>0.500                  | 0.00<br>0.00               |
| 0.6               | 1 228.8                    | 153.6                       | 154<br>153              | 0.602<br>0.59765625             | +0.26<br>−0.39             |
| 0.7               | 1 433.6                    | 179.2                       | 180<br>179              | 0.703<br>0.69921875             | +0.45<br>−0.11             |
| 0.8               | 1 638.4                    | 204.8                       | 205<br>204              | 0.801<br>0.796875               | +0.10<br>−0.39             |
| 0.9               | 1 843.2                    | 230.4                       | 231<br>230              | 0.902<br>0.8984375              | +0.26<br>−0.17             |
| 1.0               | 2 048.0                    | 256.0                       | 256<br>256              | 1.000<br>1.000                  | 0.00<br>0.00               |


In [2]:
# Please change as you need
t_DC = 10 # [s]
pps = 0 # pulses at initialization
step = 100 # [pps]
rst = 1  # [s] reset time
target_freq = 1  # [rps]
RUN_SEC = 100_000



# ------------------------------------------------------------  parameters
DEBUG = True  # set True for jitter log
SAMPLE_RATE = 100_000  # Hz
GEN_CHUNK_SEC = 0.1 # corresponds to PROC_INTERVAL up to 0.05
PROC_INTERVAL = 0.125          # process rate

CHUNK_SEC     = GEN_CHUNK_SEC
N_SAMPLES_GEN = int(SAMPLE_RATE * GEN_CHUNK_SEC) # 100_000 * 0.05 = 5000
SAMPLES_PROC = int(PROC_INTERVAL * SAMPLE_RATE) # 0.125s * 100kHz = 12500
QUEUE_DEPTH = 40  # raw AB backlog
QUAD_DEPTH = 40  # processed backlog (same)
DISPLAY_SEC = RUN_SEC+600
PLOT_SEC = 0.15  # *** fixed x-axis window width (s) *** 1/(0.2*36) < 0.15, 1/(1*36) = 0.03 in 0.03 case pruning length might be too short
GUI_INTERVAL_MS = 50  # update interval (ms) e.g. 50ms = 20Hz

NUM_PULSES = 36000
STABLE_SEC = 7200

# encoder pulse parameters
PULSE_HEIGHT = 5.0  # amplitude
INPUT_VELOCITY = 0.2  # rps

PULSE_WIDTH = 1 / (INPUT_VELOCITY * 512)  # period  (s)
PULSE_DUTY = 0.5  # duty
PULSE_PHASE_A = 0.0  # phase offset (s)
PULSE_PHASE_B = -PULSE_WIDTH / 4  # phase offset (s)

QUADPULSE_WIDTH = 0.00025  # width (s) assuming 4x given 1 rps
THRESHOLD_DEFAULT = 2.5  # logic threshold (V)

IDEAL_CPS = INPUT_VELOCITY   # counts/s (constant ideal)

# power supply parameters
AMPLITUDE = 0.3  # V
OMEGA = 2*np.pi*36*INPUT_VELOCITY

# GUI buffer lengths
PRUNING = 100  # pruning length (s) for GUI
HISTORY = int(SAMPLE_RATE * PLOT_SEC / PRUNING) # = 15000
COUNT_HISTORY = int(8*10) # <-- 1/0.125 * 10 sec
VELO_HISTORY = COUNT_HISTORY
POW_HISTORY = COUNT_HISTORY

# ------------------------------------------------------------  logging
LOG_CHUNK = 1024       # flush log queue every LOG_CHUNK messages
LOG_DATA_NUM = 4  # number of data columns in log file (t, A, B, quad)



# ------------------------------------------------------------  queues & stop flag
buf_q = queue.Queue(maxsize=QUEUE_DEPTH)  # raw (t, A, B)
quad_q = queue.Queue(maxsize=QUAD_DEPTH)  # processed (t, A, B, quad)
stop_writer = threading.Event()

log_q = queue.Queue(maxsize=0)
queue_h = logging.handlers.QueueHandler(log_q)
logger = logging.getLogger("debug")
logger.setLevel(logging.INFO)
logger.addHandler(queue_h)

In [3]:
# ------------------------------------------------------------  queues & stop flag
buf_q = queue.Queue(maxsize=QUEUE_DEPTH)  # raw (t, A, B)
quad_q = queue.Queue(maxsize=QUAD_DEPTH)  # processed (t, com, vel, pow)
stop_writer = threading.Event()

log_q = queue.Queue(maxsize=0)
queue_h = logging.handlers.QueueHandler(log_q)
logger = logging.getLogger("debug")
logger.setLevel(logging.INFO)
logger.addHandler(queue_h)

run_dir = pathlib.Path("../runs")
run_dir.mkdir(exist_ok=True)
timestamp = datetime.now().strftime("%y%m%d%H%M%S")
h5path = f"{run_dir}/{timestamp}.h5"

h5f = h5py.File(h5path, "w")
dset = h5f.create_dataset(
    "log", shape=(0, LOG_DATA_NUM), maxshape=(None, LOG_DATA_NUM),
    dtype=np.float32, chunks=(LOG_CHUNK, LOG_DATA_NUM), compression="gzip"
)

print("dset shape  =", dset.shape)
print("dset maxshape =", dset.maxshape)

dset shape  = (0, 4)
dset maxshape = (None, 4)


In [ ]:
def gen_chunk_pulse(
    t: np.ndarray,
    *,
    height: float = PULSE_HEIGHT,
    width: float = PULSE_WIDTH,
    duty: float = PULSE_DUTY,
    phase: float = 0.0,
) -> np.ndarray:
    mod = (t + phase) % width
    return np.where(mod < duty * width, height, 0.0).astype(np.float32)

def gen_chunk_sin(
    time: np.ndarray,
    *,
    A: float = AMPLITUDE,  # amplitude
    omega: float = OMEGA,  # angular frequency
    phase: float = 0.0,  # phase offset in radians
) -> np.ndarray:
    """
    Generate a sine wave chunk.

    Parameters
    ----------
    time : ndarray[float32]
        Time axis for the sine wave.
    A : float
        Amplitude of the sine wave.
    omega : float
        Angular frequency of the sine wave.
    phase : float
        Phase offset in radians.

    Returns
    -------
    ndarray[float32]
        Sine wave values at the given time points.
    """
    n = np.random.randn(len(time))  # random noise
    return A*np.sin(omega*time + phase) + A*0.01*n


REL_AXIS_GEN = (
    np.arange(N_SAMPLES_GEN, dtype=np.float32) / SAMPLE_RATE
)  # 0 ... 0.2 s

# ------------------------------------------------------------  AB → direction → quad helpers


def gen_pulse_direction(
    dA: np.ndarray,
    dB: np.ndarray,
    *,
    threshold: float,
    prev_A: bool | None = None,
    prev_B: bool | None = None,
) -> tuple[np.ndarray, bool, bool]:
    """
    Detect direction for one block **with perfect edge coverage**.

    Parameters
    ----------
    dA, dB : ndarray[float32]
        Analog levels of phase-A / phase-B for the current block.
    threshold : float
        Logic threshold [V].
    prev_A, prev_B : bool | None
        Logical state of A/B *at the end of the PREVIOUS block*.
        • If None (first block), the function falls back to the
          “self-shift” method used before.

    Returns
    -------
    dir_log : ndarray[int8]
        +1 = CW edge, –1 = CCW edge, 0 = no edge.
    last_A, last_B : bool
        Logical state of A/B at the *end* of this block — feed these
        into the next call to avoid losing the boundary edge.
    """
    # --- current logic level ----------------------------------------
    A = dA > threshold
    B = dB > threshold

    # --- previous sample for XOR -----------------------------------
    if prev_A is None:  # first block → old behaviour
        A_prev = np.concatenate(([A[0]], A[:-1]))
        B_prev = np.concatenate(([B[0]], B[:-1]))
    else:               # use states carried over from last block
        A_prev = np.concatenate(([prev_A], A[:-1]))
        B_prev = np.concatenate(([prev_B], B[:-1]))

    dir_log = (B_prev ^ A).astype(int) - (A_prev ^ B).astype(int)

    return dir_log.astype(np.int8), bool(A[-1]), bool(B[-1])

def pulse_count(dir_log: np.ndarray) -> int:
    #return int(np.sum(dir_log))
    return np.sum(dir_log)


def gen_quad_pulse(
    t: np.ndarray, dir_log: np.ndarray, width: float, height: float, sampling_rate: int
) -> np.ndarray:
    samples = int(width * sampling_rate)
    if samples <= 0:
        return np.zeros_like(dir_log, dtype=np.float32)
    base = np.full(samples, height, dtype=np.float32)
    return np.convolve(dir_log, base, mode="full")[: len(t)]


# ------------------------------------------------------------  producer thread
def generator() -> None:
    """Generate AB rectangular-wave chunks at real-time cadence."""
    chunk_idx = 0
    next_t = time.perf_counter()
    while not stop_writer.is_set():
        base = chunk_idx * GEN_CHUNK_SEC
        t_axis = REL_AXIS_GEN + base
        pulse_A = gen_chunk_pulse(t_axis, phase=PULSE_PHASE_A)
        pulse_B = gen_chunk_pulse(t_axis, phase=PULSE_PHASE_B)

        # --- add sine wave for testing purposes ---
        Iu = gen_chunk_sin(t_axis, A=0.3, omega=OMEGA, phase=0.0)
        Iv = gen_chunk_sin(t_axis, A=0.3, omega=OMEGA, phase=2*np.pi/3)
        Iw = gen_chunk_sin(t_axis, A=0.3, omega=OMEGA, phase=4*np.pi/3)
        Vu = gen_chunk_sin(t_axis, A=1.0, omega=OMEGA, phase=np.pi/8)
        Vv = gen_chunk_sin(t_axis, A=1.0, omega=OMEGA, phase=2*np.pi/3 + np.pi/8)
        Vw = gen_chunk_sin(t_axis, A=1.0, omega=OMEGA, phase=4*np.pi/3 + np.pi/8)

        try:
            buf_q.put_nowait((t_axis, pulse_A, pulse_B, Iu, Iv, Iw, Vu, Vv, Vw))
        except queue.Full:
            pass

        chunk_idx += 1
        next_t += GEN_CHUNK_SEC
        sleep = next_t - time.perf_counter()
        if sleep > 0:
            time.sleep(sleep)
        else:
            next_t = time.perf_counter()


# ------------------------------------------------------------  consumer thread
def log_listener():
    handler = logging.StreamHandler(sys.stdout)
    listener = logging.handlers.QueueListener(log_q, handler)
    listener.start()
    stop_writer.wait()
    listener.stop()

def command_freq(t_DC, pps, step, rst, num_pulse, target_freq, t_stable, proc_interval):
    num_step = np.ceil((num_pulse-pps)*target_freq/step) #stableに到達するまでの段数
    time = np.arange(0, t_DC+num_step*rst+t_stable, proc_interval, dtype=np.float32)
    freq = np.zeros(int((t_DC+num_step*rst+t_stable)*(1/proc_interval)), dtype=np.float32)
    n_DC = int(np.round(t_DC/proc_interval))  #DCの速度指令値=0rps
    freq[:n_DC] = 0.0
    n_rst = int(np.round(rst/proc_interval))  #加速中の速度指令値
    freq[n_DC: int(n_DC+n_rst*num_step)] = [pps/num_pulse + target_freq/num_pulse*step*(i+1) for i in range(int(num_step)) for j in freq[int(n_DC+i*n_rst): int(n_DC+(i+1)*n_rst)]]
    #Stableの速度指令値
    freq[int(n_DC+n_rst*num_step):] = target_freq
    return time, freq



# schmitt trigger
def schmitt_trigger(upper, lower, current):
    y_schmitt = np.zeros_like(current)
    state = 0.0
    for i, sample in enumerate(current):
        if state == 0.0 and sample >= upper:
            state = 1.0
        elif state == 1.0 and sample <= lower:
            state = 0.0
        y_schmitt[i] = state
    
    d = np.diff(y_schmitt.astype(int))      # diff（+1 なら立ち上がり, −1 なら立ち下がり）
    rise_idx = np.where(d ==  1)[0] + 1     # 立ち上がり位置
    fall_idx = np.where(d == -1)[0] + 1     # 立ち下がり位置

    return rise_idx, fall_idx

# make function to calculate power
def calc_power_refact(time, I_u, I_v, I_w, V_u, V_v, V_w, upper, lower):
    # schmitt_triggerを用いて電流の1周期につき1点を取ってくる
    rise_idx_I_u = schmitt_trigger(upper, lower, I_u)[0]
    #rise_idx_I_v = schmitt_trigger(upper, lower, I_v)[0]
    #rise_idx_I_w = schmitt_trigger(upper, lower, I_w)[0]

    # proc_intervalの中にI_uが2周期以上入るとき->初めの1周期を取り出してPを計算
    if len(rise_idx_I_u) >= 3:
        period = rise_idx_I_u[1] - rise_idx_I_u[0]
        P_u = np.mean(I_u[rise_idx_I_u[0] : rise_idx_I_u[0]+period] * V_u[rise_idx_I_u[0] : rise_idx_I_u[0]+period])
        P_v = np.mean(I_v[rise_idx_I_u[0]+int(period/3) : rise_idx_I_u[0]+int(4*period/3)] * V_v[rise_idx_I_u[0]+int(period/3) : rise_idx_I_u[0]+int(4*period/3)])
        P_w = np.mean(I_w[rise_idx_I_u[0]+int(2*period/3) : rise_idx_I_u[0]+int(5*period/3)] * V_w[rise_idx_I_u[0]+int(2*period/3) : rise_idx_I_u[0]+int(5*period/3)])
        P_tot = P_u + P_v + P_w

    # proc_intervalの中にI_uが2周期入らないとき->proc_interval全体でPを計算
    else:
        P_u = np.mean(I_u * V_u)
        P_v = np.mean(I_v * V_v)
        P_w = np.mean(I_w * V_w)
        P_tot = P_u + P_v + P_w
    
    # 中央の時間を計算
    time_p = time[0] + (time[-1] - time[0]) / 2

    return time_p,P_u,P_v,P_w,P_tot

class IdealVelocityProvider: # 0 sec からstepはいるので、想定時間よりも1rst分早くなるのが正常
    def __init__(self, t_DC, pps, step, rst, num_pulse, target_freq, t_stable, proc_interval) -> None:
        #num_points: int = int(duration / dt) + 1
        #self.time_axis: np.ndarray = np.linspace(0.0, duration, num_points, dtype=np.float32)
        #self.velocity:  np.ndarray = np.full(num_points, IDEAL_CPS, dtype=np.float32)
        self.t_DC = t_DC
        self.pps = pps
        self.step = step
        self.rst = rst
        self.num_pulse = num_pulse
        self.target_freq = target_freq
        self.t_stable = t_stable
        self.proc_interval = proc_interval
        self.time_axis, self.velocity = command_freq(t_DC, pps, step, rst, num_pulse, target_freq, t_stable, proc_interval)


    def slice(self, t_start: float, t_end: float) -> tuple[np.ndarray, np.ndarray]:
        """Return the time and velocity arrays that fall within [t_start, t_end]."""
        idx_start, idx_end = np.searchsorted(self.time_axis, [t_start, t_end])
        return self.time_axis[idx_start:idx_end], self.velocity[idx_start:idx_end]

ideal: IdealVelocityProvider | None = None  # initialised lazily in processor()


def processor() -> None:
    """
    dequeにしないとバグる
    """
    # ---------- one-time init ----------
    global ideal_provider
    if 'ideal_provider' not in globals() or ideal_provider is None:
        ideal_provider = IdealVelocityProvider(t_DC, pps, step, rst, NUM_PULSES, target_freq, STABLE_SEC, PROC_INTERVAL)

    # ---------- logging ----------
    buf = np.empty((LOG_CHUNK, LOG_DATA_NUM), dtype=np.float32)  # 内部バッファ
    buf_len = 0
    print("buf shape =", buf.shape)

    # --- Python deque リングバッファ ---
    ring_t: deque[np.float32] = deque()
    ring_a: deque[np.float32] = deque()
    ring_b: deque[np.float32] = deque()

    ring_Iu: deque[np.float32] = deque()
    ring_Iv: deque[np.float32] = deque()
    ring_Iw: deque[np.float32] = deque()
    ring_Vu: deque[np.float32] = deque()
    ring_Vv: deque[np.float32] = deque()
    ring_Vw: deque[np.float32] = deque()


    last_A = last_B = None

    next_proc = time.perf_counter()
    cum_count = 0
    last_ts   = next_proc

    while not stop_writer.is_set():
        # ---------- 非ブロッキングで出来るだけ取り込む ----------
        try:
            while True:
                t, pA, pB, Iu, Iv, Iw, Vu, Vv, Vw = buf_q.get_nowait()
                ring_t.extend(t); ring_a.extend(pA); ring_b.extend(pB)
                ring_Iu.extend(Iu); ring_Iv.extend(Iv); ring_Iw.extend(Iw)
                ring_Vu.extend(Vu); ring_Vv.extend(Vv); ring_Vw.extend(Vw)
                buf_q.task_done()
        except queue.Empty:
            pass

        # ---------- 次の処理時刻まで待機 ----------
        now = time.perf_counter()
        if now < next_proc:
            time.sleep(next_proc - now)
            continue
        next_proc += PROC_INTERVAL

        # ---------- サンプル不足ならスキップ ----------
        if len(ring_t) < SAMPLES_PROC:
            continue

        # ---------- deque → NumPy へコピー ----------
        t_blk = np.array([ring_t.popleft() for _ in range(SAMPLES_PROC)], dtype=np.float32)
        a_blk = np.array([ring_a.popleft() for _ in range(SAMPLES_PROC)], dtype=np.float32)
        b_blk = np.array([ring_b.popleft() for _ in range(SAMPLES_PROC)], dtype=np.float32)

        Iu_blk = np.array([ring_Iu.popleft() for _ in range(SAMPLES_PROC)], dtype=np.float32)
        Iv_blk = np.array([ring_Iv.popleft() for _ in range(SAMPLES_PROC)], dtype=np.float32)
        Iw_blk = np.array([ring_Iw.popleft() for _ in range(SAMPLES_PROC)], dtype=np.float32)
        Vu_blk = np.array([ring_Vu.popleft() for _ in range(SAMPLES_PROC)], dtype=np.float32)
        Vv_blk = np.array([ring_Vv.popleft() for _ in range(SAMPLES_PROC)], dtype=np.float32)
        Vw_blk = np.array([ring_Vw.popleft() for _ in range(SAMPLES_PROC)], dtype=np.float32)

        # ---------- 信号処理 ----------
        dir_log, last_A, last_B = gen_pulse_direction(
            a_blk, b_blk,
            threshold=THRESHOLD_DEFAULT,
            prev_A=last_A, prev_B=last_B
        )
        quad_sig  = gen_quad_pulse(t_blk, dir_log, QUADPULSE_WIDTH, PULSE_HEIGHT, SAMPLE_RATE)
        delta_cnt = pulse_count(dir_log)
        cum_count += delta_cnt
        velocity  = delta_cnt / PROC_INTERVAL / 2048



        # Power calculation
        time_p, P_u, P_v, P_w, P_tot = calc_power_refact(
            t_blk, Iu_blk, Iv_blk, Iw_blk,
            Vu_blk, Vv_blk, Vw_blk, 0, -0.1
        )

        # ---------- 理想速度スライス ----------
        t_ref, v_ref = ideal_provider.slice(t_blk[0], t_blk[-1])

        # ---------- ログ ----------
        '''if DEBUG:
            now = time.perf_counter()
            jitter = (now - last_ts) * 1e3
            logger.info(
                "EPOCH = %f, wall = %6.2f ms, jitter = %6.2f ms  delta c=%+d, v=%6.3f, len(dir_log)=%d, buf_len=%d",
                now, jitter, (now - last_ts) * 1e3,
                delta_cnt, velocity, len(dir_log), buf_len
            )'''

        if DEBUG:
            now = time.perf_counter()
            jitter = (now - last_ts) * 1e3
            logger.info(
                "EPOCH = %f, wall = %6.2f ms, jitter = %6.2f ms  delta c=%+d, v=%6.3f, v_ref=%6.3f, time_p = %f, P_tot = %f",
                now, jitter, (now - last_ts) * 1e3,
                delta_cnt, velocity, v_ref, time_p, P_tot
            )
            last_ts = now

        # ---------- GUI へ送信 ----------
        try:
            quad_q.put_nowait(
                (t_blk, a_blk, b_blk, quad_sig,
                 t_blk[-1], cum_count, velocity,
                 t_ref, v_ref, time_p, P_tot, Iu_blk, Vu_blk)
            )
        except queue.Full:
            pass

         # ---- append to buffer ----
        buf[buf_len] = (t_blk[-1], (v_ref[-1] if v_ref.size else 0.0), velocity, P_tot)
        buf_len += 1
        if buf_len == buf.shape[0]:
            n = dset.shape[0]
            dset.resize(n+buf_len, axis=0); dset[-buf_len:] = buf
            buf_len = 0

     # ---- final flush ----
    if buf_len:
        n = dset.shape[0]
        dset.resize(n+buf_len, axis=0); dset[-buf_len:] = buf[:buf_len]
    h5f.close()


def start_gui() -> None:
    pg.setConfigOptions(useOpenGL=True, background="w", foreground="k")
    app = pg.mkQApp("Live plots")

    win = pg.GraphicsLayoutWidget(title="DEMO")

    layout = win.ci.layout                 # GraphicsLayout の中身
    layout.setColumnStretchFactor(0, 4)
    layout.setColumnStretchFactor(1, 5)
    #win_sig = pg.GraphicsLayoutWidget(show=True, title="Signal")
    #win_vel = pg.GraphicsLayoutWidget(show=True, title="Velocity")

    #win_sig.resize(800, 600)
    #win_vel.resize(800, 600)
    #win_sig.show()
    pruning = PRUNING
    if pruning < 1:
        pruning = 1 # 0以下の値は無効とし、1に設定


    win.resize(800, 600)
    win.show()

    # [0, 0] A/B -------------------------------------------------------
    plt_ab = win.addPlot(row=0, col=0, title="RAW A / B")
    curve_A = plt_ab.plot(pen=pg.mkPen("#ff4b00", width=3),stepMode="right")
    curve_B = plt_ab.plot(pen=pg.mkPen("#005aff", width=3),stepMode="right")
    plt_ab.setLabel("left", "Amplitude [V]")
    plt_ab.setLabel("bottom", "Time [s]")
    plt_ab.setYRange(-0.5, PULSE_HEIGHT + 0.5)

    # [1,0] Quad waveform --------------------------------------------
    #plt_q = win.addPlot(row=1, col=0, title="Quad pulse")
    #curve_Q = plt_q.plot(pen=pg.mkPen("m", width=3))
    #plt_q.setLabel("left", "Amplitude [V]")
    #plt_q.setLabel("bottom", "Time [s]")
    #plt_q.setYRange(-PULSE_HEIGHT - 0.5, PULSE_HEIGHT + 0.5)

    # [2,0] I/V waveform --------------------------------------------
    plt_IV = win.addPlot(row=2, col=0, title="PhaseU I / V")
    curve_I = plt_IV.plot(pen=pg.mkPen("r", width=3))
    curve_V = plt_IV.plot(pen=pg.mkPen("b", width=3))
    plt_IV.setLabel("left", "Amplitude [a.u.]")
    plt_IV.setLabel("bottom", "Time [s]")
    plt_IV.setYRange(-1.2, 1.2)

    # [0,1] count (fixed x-axis) -------------------------------------
    plt_cnt = win.addPlot(row=0, col=1, title="Velovity - Command")
    curve_cnt = plt_cnt.plot(pen=pg.mkPen("#03af7a", width=3))
    #plt_cnt.setXRange(0, RUN_SEC, padding=0)
    #plt_cnt.enableAutoRange("x", True)
    plt_cnt.setLabel("left", "Diff")
    plt_cnt.setLabel("bottom", "Time [s]")

    # [1,1] velocity
    plt_vel = win.addPlot(row=1, col=1, title="Velocity")
    curve_vel     = plt_vel.plot(pen=pg.mkPen("#00a0e9", width=3))   # measured
    curve_vel_ref = plt_vel.plot(pen=pg.mkPen("#a05aff", width=3), stepMode="right")   # ideal (new)
    #plt_vel.setXRange(0, RUN_SEC, padding=0)
    #plt_vel.enableAutoRange("x", True)
    plt_vel.setLabel("left", "Velocity [rps]")
    plt_vel.setLabel("bottom", "Time [s]")

    # [2,1] power ----------------------------------
    plt_pow = win.addPlot(row=2, col=1, title="Power")
    curve_pow     = plt_pow.plot(pen=pg.mkPen("#f6aa00", width=3))   # measured
    plt_pow.setLabel("left", "Power [W]")
    plt_pow.setLabel("bottom", "Time [s]")

    # buffers ---------------------------------------------------------
    xs = ya = yb = yq = np.empty(0, dtype=np.float32)
    xs_cnt = y_cnt = np.empty(0, dtype=np.float32)
    xs_vel = y_vel = np.empty(0, dtype=np.float32)
    xr = yr = np.empty(0, dtype=np.float32)       # ideal velocity buffers (new)
    y_Iu = y_Vu = np.empty(0, dtype=np.float32)  # I/V buffers (new)
    xs_time_p = y_P_tot = np.empty(0, dtype=np.float32)  # time and power buffers (new)

    def refresh():
        nonlocal xs, ya, yb, yq, xs_cnt, y_cnt, xs_vel, y_vel, xr, yr, xs_time_p, y_P_tot, y_Iu, y_Vu
        try:
            while True:
                # receive processed data
                t_ax, pA, pB, qsig, t_end, cum_cnt, vel, t_ref, v_ref, time_p, P_tot, Iu_blk, Vu_blk = quad_q.get_nowait()

                xs = np.concatenate((xs, t_ax[::pruning]))[-HISTORY:]
                ya = np.concatenate((ya, pA[::pruning]))[-HISTORY:]
                yb = np.concatenate((yb, pB[::pruning]))[-HISTORY:]
                #yq = np.concatenate((yq, qsig))[-HISTORY:]

                xs_cnt = np.append(xs_cnt, t_end)[-COUNT_HISTORY:]
                y_cnt  = np.append(y_cnt, vel-v_ref)[-COUNT_HISTORY:]

                xs_vel = np.append(xs_vel, t_end)[-VELO_HISTORY:]
                y_vel  = np.append(y_vel, vel)[-VELO_HISTORY:]

                xr = np.concatenate((xr, t_ref))[-VELO_HISTORY:]
                yr = np.concatenate((yr, v_ref))[-VELO_HISTORY:]

                y_Iu = np.concatenate((y_Iu, Iu_blk[::pruning]))[-HISTORY:]
                y_Vu = np.concatenate((y_Vu, Vu_blk[::pruning]))[-HISTORY:]

                xs_time_p = np.append(xs_time_p, time_p)[-POW_HISTORY:]
                y_P_tot = np.append(y_P_tot, P_tot)[-POW_HISTORY:]

                quad_q.task_done()
        except queue.Empty:
            pass

        # scrolling window for waveforms only
        if xs.size:
            start = xs[-1] - PLOT_SEC
            plt_ab.setXRange(start, xs[-1], padding=0)
            plt_IV.setXRange(start, xs[-1], padding=0)

        # --- auto-range for count/velocity ---
        if xs_cnt.size:
            plt_cnt.setXRange(xs_cnt[-1]-10, xs_cnt[-1], padding=0)

        if xs_vel.size:
            plt_vel.setXRange(xs_vel[-1]-10, xs_vel[-1], padding=0)

        if xs_time_p.size:
            plt_pow.setXRange(xs_time_p[-1]-10, xs_time_p[-1], padding=0)

        # --- push data to curves ---
        curve_A.setData(xs, ya)
        curve_B.setData(xs, yb)
        curve_I.setData(xs, y_Iu)
        curve_V.setData(xs, y_Vu)
        curve_cnt.setData(xs_cnt, y_cnt)
        curve_vel.setData(xs_vel, y_vel)
        curve_vel_ref.setData(xr, yr)
        curve_pow.setData(xs_time_p, y_P_tot)

    timer = QtCore.QTimer()
    timer.timeout.connect(refresh)
    timer.start(GUI_INTERVAL_MS)

    # auto-stop after RUN_SEC
    QtCore.QTimer.singleShot(int(RUN_SEC * 1000), lambda: (stop_writer.set(), app.quit()))
    app.exec()


In [5]:
if __name__ == "__main__":
    threading.Thread(target=log_listener, daemon=True).start()  # start log listener

    gen_th  = threading.Thread(target=generator, daemon=True)
    proc_th = threading.Thread(target=processor, daemon=True)

    gen_th.start()
    proc_th.start()
    # con_th.start()

    start_gui()  # blocks until the user closes the window or timer expires

    # join threads and exit
    stop_writer.set()
    gen_th.join()
    proc_th.join()

    print("Graceful shutdown.")


buf shape = (1024, 4)
EPOCH = 867431.668332, wall = 1357.35 ms, jitter = 1357.35 ms  delta c=+51, v= 0.199, v_ref= 0.000, time_p = 0.062495, P_tot = 0.415717
EPOCH = 867431.765468, wall =  97.14 ms, jitter =  97.14 ms  delta c=+51, v= 0.199, v_ref= 0.000, time_p = 0.187495, P_tot = 0.415754
EPOCH = 867431.869085, wall = 103.62 ms, jitter = 103.62 ms  delta c=+51, v= 0.199, v_ref= 0.000, time_p = 0.312495, P_tot = 0.415761
EPOCH = 867432.042673, wall = 173.59 ms, jitter = 173.59 ms  delta c=+51, v= 0.199, v_ref= 0.000, time_p = 0.437495, P_tot = 0.415783
EPOCH = 867432.123576, wall =  80.90 ms, jitter =  80.90 ms  delta c=+51, v= 0.199, v_ref= 0.000, time_p = 0.562495, P_tot = 0.415724
EPOCH = 867432.272708, wall = 149.13 ms, jitter = 149.13 ms  delta c=+52, v= 0.203, v_ref= 0.000, time_p = 0.687495, P_tot = 0.415727
EPOCH = 867432.385561, wall = 112.85 ms, jitter = 112.85 ms  delta c=+51, v= 0.199, v_ref= 0.000, time_p = 0.812495, P_tot = 0.415801
EPOCH = 867432.506216, wall = 120.65 m